<a href="https://colab.research.google.com/github/adibg17/TOON_EMOTION_CLASSIFER-Using-MobilenetV2/blob/main/TOON_EMOTION_MobilenetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt


In [ ]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')


In [ ]:
dataset_path = "/content/drive/MyDrive/TOON_EMOTION"
character_map = {"Tom": 0, "Jerry": 1}
emotion_map = {"Angry": 0, "Happy": 1, "Sad": 2, "Surprised": 3}

image_paths, character_labels, emotion_labels = [], [], []

for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        character = "Tom" if "Tom" in folder else "Jerry"
        emotion = folder.replace(f"_{character}", "")
        char_id = character_map[character]
        emo_id = emotion_map[emotion]

        for file in os.listdir(folder_path):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(folder_path, file))
                character_labels.append(char_id)
                emotion_labels.append(emo_id)


In [ ]:
data = list(zip(image_paths, character_labels, emotion_labels))
np.random.seed(42)
np.random.shuffle(data)
image_paths, character_labels, emotion_labels = zip(*data)

image_paths = np.array(image_paths)
character_labels = tf.keras.utils.to_categorical(character_labels, 2)
emotion_labels = tf.keras.utils.to_categorical(emotion_labels, 4)


In [ ]:
train_paths, val_paths, train_char, val_char, train_emo, val_emo = train_test_split(
    image_paths, character_labels, emotion_labels, test_size=0.2, random_state=42
)


In [ ]:
img_size = (128, 128)
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(path, char_label, emo_label):
    try:
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, img_size)
        img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    except:
        img = tf.zeros((128, 128, 3))  # Fallback if image fails
    return img, {'char_output': char_label, 'emo_output': emo_label}

def create_dataset(paths, char_labels, emo_labels):
    ds = tf.data.Dataset.from_tensor_slices((paths, char_labels, emo_labels))
    ds = ds.map(preprocess, num_parallel_calls=AUTOTUNE)
    ds = ds.shuffle(500).batch(batch_size).prefetch(AUTOTUNE)
    return ds

train_ds = create_dataset(train_paths, train_char, train_emo)
val_ds = create_dataset(val_paths, val_char, val_emo)


In [ ]:
for x, y in train_ds.take(1):
    print("Batch shape:", x.shape)
    print("Char labels:", y['char_output'][0])
    print("Emo labels:", y['emo_output'][0])


Batch shape: (32, 128, 128, 3)
Char labels: tf.Tensor([1. 0.], shape=(2,), dtype=float64)
Emo labels: tf.Tensor([0. 0. 1. 0.], shape=(4,), dtype=float64)


In [ ]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)

char_output = Dense(2, activation='softmax', name='char_output')(x)
emo_output = Dense(4, activation='softmax', name='emo_output')(x)

model = Model(inputs=base_model.input, outputs=[char_output, emo_output])
model.compile(optimizer='adam',
              loss={'char_output': 'categorical_crossentropy', 'emo_output': 'categorical_crossentropy'},
              metrics={'char_output': 'accuracy', 'emo_output': 'accuracy'})

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast (Cast)         │ (None, 128, 128,  │          0 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ cast[0][0]        │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis

 Total params: 2,265,670 (8.64 MB)

 Trainable params: 7,686 (30.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=5)


Epoch 1/5
205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - char_output_accuracy: 0.8431 - char_output_loss: 0.3662 - emo_output_accuracy: 0.3399 - emo_output_loss: 1.8163 - loss: 2.1825

In [ ]:
from sklearn.metrics import classification_report

characters = ['Tom', 'Jerry']
emotions = ['Angry', 'Happy', 'Sad', 'Surprised']

def get_labels_and_preds(dataset):
    y_true_char, y_pred_char = [], []
    y_true_emo, y_pred_emo = [], []

    for x, y in dataset:
        preds = model.predict(x, verbose=0)
        y_true_char.extend(np.argmax(y['char_output'], axis=1))
        y_pred_char.extend(np.argmax(preds[0], axis=1))
        y_true_emo.extend(np.argmax(y['emo_output'], axis=1))
        y_pred_emo.extend(np.argmax(preds[1], axis=1))

    return y_true_char, y_pred_char, y_true_emo, y_pred_emo

y_true_char, y_pred_char, y_true_emo, y_pred_emo = get_labels_and_preds(val_ds)

print("Character Classification Report:")
print(classification_report(y_true_char, y_pred_char, target_names=characters))

print("Emotion Classification Report:")
print(classification_report(y_true_emo, y_pred_emo, target_names=emotions))


In [ ]:
def predict_image(img_path):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    char_probs, emo_probs = model.predict(img_array)

    char_idx = np.argmax(char_probs)
    emo_idx = np.argmax(emo_probs)

    char_pred = characters[char_idx]
    emo_pred = emotions[emo_idx]

    print(f"Predicted Character: {char_pred} ({np.max(char_probs)*100:.2f}%)")
    print(f" Predicted Emotion : {emo_pred} ({np.max(emo_probs)*100:.2f}%)")

    plt.imshow(image.load_img(img_path))
    plt.title(f"{char_pred} - {emo_pred}")
    plt.axis('off')
    plt.show()



In [ ]:
predict_image("/content/drive/MyDrive/TOON_EMOTION/download.jpg")


In [ ]:
model.save("/content/drive/MyDrive/tom_jerry_emotion_model.h5")


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/tom_jerry_emotion_model.h5")
#TO LOAD LATER

In [ ]:
predict_image("/content/drive/MyDrive/TOON_EMOTION/Surprised_Jerry/Jerry_Surprised_997.png")